In [1]:
import json
import pandas as pd
import seaborn as sns
from databases.mongodb import MongoDB
from databases.blockchain_etl import BlockchainETL
from databases.postgresql import PostgresDB
from copy import deepcopy
from itertools import combinations

DB_PREFIX = ''
with open('./artifacts/centralized_exchange_addresses.json', 'r') as f:
    cex_hot_wallets = json.load(f)


cexes = ['binance', 'coinbase-exchange', 'kucoin', 'bitfinex', 'crypto-com-exchange', 'huobi', 'bybit', 'gate-io', 'binance-us', 'okx', 'mxc', 'bilaxy']
dexes = ["pancakeswap", "spookyswap"]
lending_pools_0 = ['aave', 'venus', 'trava', 'cream', 'valas', 'compound', 'geist']

chain_ids = ['0x38', '0x1', '0xfa', '0x89']
chain_names = ['BSC', 'Ethereum', 'Fantom', "Polygon"]

mongodb = MongoDB()
blockchain_etl = BlockchainETL(db_prefix=DB_PREFIX)
postgres = PostgresDB()

## Lending pools

Get number of lending wallets of each lending pool and on each chain

In [2]:
data = {}
columns = []

for lending in lending_pools_0:
    data[lending] = []

    for chain_id in chain_ids:
        c = mongodb.count_deployed_chains_of_project('lendingpools', lending, chain_id)
        c += mongodb.count_deployed_chains_of_project('lendingpools', 'pool_' + lending, chain_id)
        data[lending].append(c)
    _filter_1 = {f"lendingpools.{lending}": {"$exists": True}}
    _filter_2 = {f"lendingpools.pool_{lending}": {"$exists": True}}
    sum = mongodb.count_wallets(_filter_1) + mongodb.count_wallets(_filter_2)
    data[lending].append(sum)

columns=deepcopy(chain_names)
columns.insert(0, 'sum')
print(data)
df = pd.DataFrame.from_dict(data, orient='index', columns=columns)
df

{'aave': [72175, 0, 70233, 0, 1942], 'venus': [75445, 75445, 0, 0, 0], 'trava': [4400, 705, 3488, 207, 0], 'cream': [2652, 1908, 0, 744, 0], 'valas': [2154, 2154, 0, 0, 0], 'compound': [195123, 0, 195123, 0, 0], 'geist': [21857, 0, 0, 21857, 0]}


,sum,BSC,Ethereum,Fantom,Polygon
aave,72175,0,70233,0,1942
venus,75445,75445,0,0,0
trava,4400,705,3488,207,0
cream,2652,1908,0,744,0
valas,2154,2154,0,0,0
compound,195123,0,195123,0,0
geist,21857,0,0,21857,0


In [12]:
print(columns)
print(data)

['0x38', '0x1', '0xfa', '0x89', 'sum']
{'aave': [0, 70233, 0, 1942, 0, 0, 0, 72175], 'venus': [75446, 0, 0, 0, 0, 0, 0, 75446], 'trava': [705, 3488, 207, 0, 0, 0, 0, 4400], 'cream': [1908, 0, 744, 0, 0, 0, 0, 2652], 'valas': [2154, 0, 0, 0, 0, 0, 0, 2154], 'compound': [0, 195123, 0, 0, 0, 0, 0, 195123], 'geist': [0, 0, 21857, 0, 0, 0, 0, 21857]}


## Deposit wallets

In [5]:
data = {}

for cex in cexes:
    data[cex] = []
    for chain_id in chain_ids:
        c = mongodb.count_exchange_deposit_wallets_each_chain('depositedExchanges', cex, chain_id)
        data[cex].append(c)
    # total number of wallets on all chains
    _filter = {f"depositedExchanges.{cex}": {"$exists": 1}}
    _count = mongodb.count_wallets(_filter)
    data[cex].append(_count)

columns = deepcopy(chain_names)
columns.append('total')
print(data)
print(columns)

{'binance': [2610363, 3045233, 0, 0, 5040303], 'coinbase-exchange': [0, 2698378, 0, 0, 2698378], 'kucoin': [0, 1268972, 0, 0, 1268972], 'bitfinex': [0, 14979, 1088, 2130, 17974], 'crypto-com-exchange': [15250, 1101779, 9869, 68254, 1143158], 'huobi': [0, 213019, 0, 29694, 240667], 'bybit': [247029, 382956, 0, 61142, 614871], 'gate-io': [150176, 179221, 11187, 45011, 352828], 'binance-us': [0, 139271, 0, 0, 139271], 'okx': [0, 417555, 0, 227628, 613912], 'mxc': [287189, 213882, 0, 0, 481691], 'bilaxy': [0, 0, 0, 0, 0]}
['BSC', 'Ethereum', 'Fantom', 'Polygon', 'total']


In [6]:
print(data)
print(columns)
df = pd.DataFrame.from_dict(data, orient='index', columns=columns)
df


{'binance': [2610363, 3045233, 0, 0, 5040303], 'coinbase-exchange': [0, 2698378, 0, 0, 2698378], 'kucoin': [0, 1268972, 0, 0, 1268972], 'bitfinex': [0, 14979, 1088, 2130, 17974], 'crypto-com-exchange': [15250, 1101779, 9869, 68254, 1143158], 'huobi': [0, 213019, 0, 29694, 240667], 'bybit': [247029, 382956, 0, 61142, 614871], 'gate-io': [150176, 179221, 11187, 45011, 352828], 'binance-us': [0, 139271, 0, 0, 139271], 'okx': [0, 417555, 0, 227628, 613912], 'mxc': [287189, 213882, 0, 0, 481691], 'bilaxy': [0, 0, 0, 0, 0]}
['BSC', 'Ethereum', 'Fantom', 'Polygon', 'total']


,BSC,Ethereum,Fantom,Polygon,total
binance,2610363,3045233,0,0,5040303
coinbase-exchange,0,2698378,0,0,2698378
kucoin,0,1268972,0,0,1268972
bitfinex,0,14979,1088,2130,17974
crypto-com-exchange,15250,1101779,9869,68254,1143158
huobi,0,213019,0,29694,240667
bybit,247029,382956,0,61142,614871
gate-io,150176,179221,11187,45011,352828
binance-us,0,139271,0,0,139271
okx,0,417555,0,227628,613912


In [5]:
cexes_pairs = combinations(cexes, 2)
addrs_set = set()
for _pair in cexes_pairs:
    _filter = {f"depositedExchanges.{_pair[0]}": {"$exists": 1}, 
               f"depositedExchanges.{_pair[1]}": {"$exists": 1}}
    _projection = {"address": 1}
    # _count = mongodb.count_wallets(_filter)
    # print(f'{_pair}: {_count}')
    data = mongodb.get_wallets(_filter)

    for datum in data:
        addr = datum['address']


KeyboardInterrupt: 

In [3]:
example_wallet = "0x4c9df57276dc17dee5635ded208c07b0be32afd0"

In [9]:
chain_id = "0x38"
cex = "binance"
print("CEX 1")
binance_wallets = cex_hot_wallets.get(cex).get('wallets').get(chain_id)
transactions = blockchain_etl.get_all_transactions_from_1_to_n(from_address=example_wallet, to_addresses=binance_wallets)
transfers = postgres.get_all_transfers_from_1_to_n(from_address=example_wallet, to_addresses=binance_wallets)
for tx in transactions:
    print(tx)
for t in transfers:
    print(t)


[05-19-2023 01:27:58 +07] [DEBUG] [Time Execute] - DATABASE:BlockchainETL.get_all_transactions_from_1_to_n executed in 0.0s


CEX 1


KeyboardInterrupt: 

In [6]:
chain_id = "0xfa"
cex = "crypto-com-exchange"
hot_wallets = cex_hot_wallets.get(cex).get('wallets').get(chain_id)
# transactions = BlockchainETL(db_prefix='ftm').get_all_transactions_from_1_to_n(from_address=example_wallet, to_addresses=hot_wallets)
transfers = postgres.get_all_transfers_from_1_to_n(from_address=example_wallet, to_addresses=hot_wallets)
for tx in transactions:
    print(tx)
for t in transfers:
    print(t)

InternalError: (psycopg2.errors.InFailedSqlTransaction) current transaction is aborted, commands ignored until end of transaction block

[SQL: 
            SELECT transaction_hash, to_address
            FROM chain_0xfa.token_transfer
            WHERE from_address = '0x4c9df57276dc17dee5635ded208c07b0be32afd0'
            AND to_address = ANY (ARRAY['0x6262998ced04146fa42253a5c0af90ca02dfd2a3', '0x72a53cdbbcc1b9efa39c834a540550e23463aacb', '0x7758e507850da48cd47df1fb5f875c23e3340c50', '0xcffad3200574698b78f32232aa9d63eabd290703'])
        ]
(Background on this error at: https://sqlalche.me/e/20/2j85)

## Dex Traders

In [3]:
c = MongoDB().count_deployed_chains_of_project('lendingpools', 'venus', '0x38')
dexes = ["pancakeswap", "spookyswap"]
for dex in dexes:
    _filter = {f"tradedlps.{dex}": {"$exists": 1}}
    print(f"{dex}: {mongodb.count_wallets(_filter)}")

pancakeswap: 18039
spookyswap: 16275
